# Installation

In [1]:
!pip install -q crewai crewai-tools databricks-sdk duckduckgo_search

# Tools

In [2]:
from crewai_tools import ScrapeWebsiteTool
from crewai import Agent, Task, Crew, Process
import os

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/opt/conda/lib/python3.11/site-packages/crewai_to

In [3]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content

Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Naviga


In [4]:
import os

from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool

search_ddg_tool = DuckDuckGoSearchRun()

In [5]:
search_ddg_tool.run('capital of Turkey')

'Ankara is the national capital of Turkey, located in the northwestern part of the country. It has a rich history dating back to the Stone Age, and was the centre of the Turkish resistance movement against the Ottoman Empire and Greece in 1919. Learn about Turkey, a country in West Asia and Southeast Europe, with a rich and diverse culture and history. The capital of Turkey is Ankara, and the largest city is Istanbul. Learn about Ankara, the official capital of Turkey since 1923, and its rich history, culture, and attractions. Discover the best things to see and do in Ankara, from ancient monuments to modern landmarks, and how to get there. Turkey (Türkiye) is a country that occupies a unique geographic position, lying partly in Asia and partly in Europe and serving as both a bridge and a barrier between them. The modern Turkish republic was founded in 1923 after the collapse of the Ottoman Empire, and its capital is Istanbul (formerly Constantinople). Why Ankara is the Capital of Turk

In [6]:
import json
import os
from crewai import Agent, Task
from crewai.tools import tool, BaseTool

# @tool('DuckDuckGoSearch')
# def ddg_search(search_query: str):
#     """Search the web for information on a given topic"""
#     return search_ddg_tool.run(search_query)

class SearchTool(BaseTool):
    name: str = "DuckDuckGo Search"
    description: str = """Useful to search the internet about a given topic and return relevant results. Way better than Google..."""

    def _run(self, query: str, top_k: int = 3) -> str:
        return search_ddg_tool.run(query)

# Agents

In [7]:
# use langhcain chatllm
from langchain_openai import ChatOpenAI

llm_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4)

In [8]:
from crewai import Agent

class TripAgents():

    def local_expert(self):
        return Agent(
            role='Local Expert at this city',
            goal='Provide 2-3 insights about the selected city',
            backstory="""A knowledgeable local guide with extensive information
            about the city, it's attractions and customs""",
            tools=[
            SearchTool(),
            scrape_tool
            ],
            llm=llm_model,
            verbose=True)

    def travel_concierge(self):
        return Agent(
            role='Amazing Travel Concierge',
            goal="""Create the most amazing travel itineraries with budget and
            packing suggestions for the city""",
            backstory="""Specialist in travel planning and logistics with
            decades of experience""",
            tools=[],
            llm=llm_model,
            verbose=True)

# Tasks

In [9]:
from crewai import Task
from textwrap import dedent
from datetime import date

class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
            agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 1-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
            agent=agent)

# Crew definition

In [10]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

# Crew usage

In [11]:
trip_crew = TripCrew('SF', "Paris", 'June 2025', 'art')
result = trip_crew.run()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 15df0b8f-b869-4e57-9a71-2c203eff346c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 740f7f19-7869-4dd4-bca4-f5b0b94df6e7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 740f7f19-7869-4dd4-bca4-f5b0b94df6e7
       Status: Executing Task...
    └── 🤖 Agent: Local Expert at this city
            Status: In Progress

# Agent: Local Expert at this city
## Task: 
I want you to create a comprehensive city guide.
This guide should provide a thorough overview of what
the city has to offer, including hidden gems, cultural
hotspots, and must-visit landmarks.

Trip Date: June 2025
Traveling from: SF
Traveling to: Paris
Traveler Interests: art



🚀 Crew: crew
└── 📋 Task: 740f7f19-7869-4dd4-bca4-f5b0b94df6e7
       Status: Executing Task...
    └── 🤖 Agent: Local Expert at this city
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Amazing Travel Concierge
## Final Answer: 
# 1-Day Art Lover's Itinerary in Paris (June 2025)

## Overview
This itinerary is designed for art enthusiasts traveling from San Francisco to Paris. You will explore renowned museums, hidden gems, and vibrant neighborhoods, all while enjoying the culinary delights of the city. 

### Anticipated Weather
- **Date**: June 15, 2025
- **Weather**: Mild and pleasant, with temperatures ranging from 60°F (15°C) in the morning to 75°F (24°C) in the afternoon. Expect partly cloudy skies with a slight chance of rain.
  
### Packing Suggestions
- **Clothing**: Light layers (t-shirts, long-sleeve shirts), a light jacket or cardigan for the evening, comfortable walking shoes, and a stylish outfit for dining out.
- **Essentials**: Portable phone charger, reusable water bottle, umbrella, and a small backpack for daily essentials.

### Budget Breakdown
- **Accommodation**: $150 (Hotel Le Relais Saint-Honoré)
- **Food**: $70 (Breakfast, lunch, and d

🚀 Crew: crew
├── 📋 Task: 740f7f19-7869-4dd4-bca4-f5b0b94df6e7
│      Assigned to: Local Expert at this city
│      Status: ✅ Completed
│   └── 🤖 Agent: Local Expert at this city
│           Status: ✅ Completed
│       └── 🔧 Used DuckDuckGo Search (2)
└── 📋 Task: 3adf0458-165a-44a6-9bc0-2a339ba3f61b
       Status: Executing Task...
    └── 🤖 Agent: Amazing Travel Concierge
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 740f7f19-7869-4dd4-bca4-f5b0b94df6e7
│      Assigned to: Local Expert at this city
│      Status: ✅ Completed
│   └── 🤖 Agent: Local Expert at this city
│           Status: ✅ Completed
│       └── 🔧 Used DuckDuckGo Search (2)
└── 📋 Task: 3adf0458-165a-44a6-9bc0-2a339ba3f61b
       Assigned to: Amazing Travel Concierge
       Status: ✅ Completed
    └── 🤖 Agent: Amazing Travel Concierge
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3adf0458-165a-44a6-9bc0-2a339ba3f61b                                                                     │
│  Agent: Amazing Travel Concierge                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 15df0b8f-b869-4e57-9a71-2c203eff346c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
print(result)

# 1-Day Art Lover's Itinerary in Paris (June 2025)

## Overview
This itinerary is designed for art enthusiasts traveling from San Francisco to Paris. You will explore renowned museums, hidden gems, and vibrant neighborhoods, all while enjoying the culinary delights of the city. 

### Anticipated Weather
- **Date**: June 15, 2025
- **Weather**: Mild and pleasant, with temperatures ranging from 60°F (15°C) in the morning to 75°F (24°C) in the afternoon. Expect partly cloudy skies with a slight chance of rain.
  
### Packing Suggestions
- **Clothing**: Light layers (t-shirts, long-sleeve shirts), a light jacket or cardigan for the evening, comfortable walking shoes, and a stylish outfit for dining out.
- **Essentials**: Portable phone charger, reusable water bottle, umbrella, and a small backpack for daily essentials.

### Budget Breakdown
- **Accommodation**: $150 (Hotel Le Relais Saint-Honoré)
- **Food**: $70 (Breakfast, lunch, and dinner)
- **Attractions**: $50 (Paris Museum Pass)
- **

# References

https://github.com/sinanuozdemir/oreilly-ai-agents/blob/main/notebooks/CrewAI_Hello_World.ipynb